In [8]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- CONFIG ---
# Base working directory
base_dir = "/content/drive/Shared drives/VisualCompGroupProject"

# Dataset path: just change this to test a different weather effect!
dataset_name = "custom_foggy_25.npy"  # Change to: custom_rainy.npy, custom_snowy.npy
dataset_path = os.path.join(base_dir, "SimpleMethodAugmentation", dataset_name)

# Path to model
model_path = os.path.join(base_dir, "resnet50_model.pth")

# Path to labels (must be aligned with test data)
y_test_path = os.path.join(base_dir, "SimpleMethodAugmentation", "labels_foggy_25.npy")  # Assumes labels are the same

# --- Dataset Wrapper ---
class NumpyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = int(self.labels[idx])
        return img, label

# --- Preprocessing ---
# These values are standard for ImageNet (which ResNet50 expects)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# --- Load Data ---
X_test = np.load(dataset_path)
y_test = np.load(y_test_path)

# Ensure dtype is correct
X_test = X_test.astype(np.uint8)

# Create DataLoader
test_dataset = NumpyDataset(X_test, y_test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 and match output classes
model = models.resnet50(pretrained=False)
num_classes = len(np.unique(y_test))
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# --- Evaluation ---
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc=f"Evaluating on {dataset_name}"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"\nEvaluation Metrics for {dataset_name}")
print(f"Accuracy : {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall   : {recall * 100:.2f}%")
print(f"F1 Score : {f1 * 100:.2f}%")

# Optional: print full classification report
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating on custom_foggy_25.npy: 100%|██████████| 395/395 [00:19<00:00, 20.42it/s]


Evaluation Metrics for custom_foggy_25.npy
Accuracy : 97.69%
Precision: 97.76%
Recall   : 97.69%
F1 Score : 97.69%

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        60
           1       0.99      0.99      0.99       720
           2       1.00      0.99      0.99       750
           3       0.98      0.93      0.96       450
           4       0.99      0.99      0.99       660
           5       0.94      0.99      0.97       630
           6       0.99      0.95      0.97       150
           7       0.98      0.95      0.97       450
           8       0.99      0.96      0.98       450
           9       0.98      1.00      0.99       480
          10       0.99      0.99      0.99       660
          11       0.96      0.97      0.96       420
          12       1.00      1.00      1.00       690
          13       1.00      1.00      1.00       720
          14       0.99      0.99      0

In [9]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- CONFIG ---
# Base working directory
base_dir = "/content/drive/Shared drives/VisualCompGroupProject"

# Dataset path: just change this to test a different weather effect!
dataset_name = "custom_foggy_50.npy"  # Change to: custom_rainy.npy, custom_snowy.npy
dataset_path = os.path.join(base_dir, "SimpleMethodAugmentation", dataset_name)

# Path to model
model_path = os.path.join(base_dir, "resnet50_model.pth")

# Path to labels (must be aligned with test data)
y_test_path = os.path.join(base_dir, "SimpleMethodAugmentation", "labels_foggy_50.npy")  # Assumes labels are the same

# --- Dataset Wrapper ---
class NumpyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = int(self.labels[idx])
        return img, label

# --- Preprocessing ---
# These values are standard for ImageNet (which ResNet50 expects)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# --- Load Data ---
X_test = np.load(dataset_path)
y_test = np.load(y_test_path)

# Ensure dtype is correct
X_test = X_test.astype(np.uint8)

# Create DataLoader
test_dataset = NumpyDataset(X_test, y_test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 and match output classes
model = models.resnet50(pretrained=False)
num_classes = len(np.unique(y_test))
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# --- Evaluation ---
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc=f"Evaluating on {dataset_name}"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"\nEvaluation Metrics for {dataset_name}")
print(f"Accuracy : {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall   : {recall * 100:.2f}%")
print(f"F1 Score : {f1 * 100:.2f}%")

# Optional: print full classification report
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating on custom_foggy_50.npy: 100%|██████████| 395/395 [00:23<00:00, 16.96it/s]



Evaluation Metrics for custom_foggy_50.npy
Accuracy : 96.42%
Precision: 96.61%
Recall   : 96.42%
F1 Score : 96.42%

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        60
           1       0.99      0.99      0.99       720
           2       0.99      0.98      0.99       750
           3       0.95      0.92      0.94       450
           4       0.99      0.98      0.98       660
           5       0.93      0.99      0.96       630
           6       0.99      0.89      0.94       150
           7       0.98      0.93      0.96       450
           8       1.00      0.92      0.96       450
           9       0.97      0.99      0.98       480
          10       0.98      0.98      0.98       660
          11       0.94      0.94      0.94       420
          12       1.00      0.99      1.00       690
          13       1.00      1.00      1.00       720
          14       0.97      0.99      0

# Snow

In [10]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- CONFIG ---
# Base working directory
base_dir = "/content/drive/Shared drives/VisualCompGroupProject"

# Dataset path: just change this to test a different weather effect!
dataset_name = "custom_snowy_25.npy"  # Change to: custom_rainy.npy, custom_snowy.npy
dataset_path = os.path.join(base_dir, "SimpleMethodAugmentation", dataset_name)

# Path to model
model_path = os.path.join(base_dir, "resnet50_model.pth")

# Path to labels (must be aligned with test data)
y_test_path = os.path.join(base_dir, "SimpleMethodAugmentation", "labels_snowy_25.npy")  # Assumes labels are the same

# --- Dataset Wrapper ---
class NumpyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = int(self.labels[idx])
        return img, label

# --- Preprocessing ---
# These values are standard for ImageNet (which ResNet50 expects)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# --- Load Data ---
X_test = np.load(dataset_path)
y_test = np.load(y_test_path)

# Ensure dtype is correct
X_test = X_test.astype(np.uint8)

# Create DataLoader
test_dataset = NumpyDataset(X_test, y_test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 and match output classes
model = models.resnet50(pretrained=False)
num_classes = len(np.unique(y_test))
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# --- Evaluation ---
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc=f"Evaluating on {dataset_name}"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"\nEvaluation Metrics for {dataset_name}")
print(f"Accuracy : {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall   : {recall * 100:.2f}%")
print(f"F1 Score : {f1 * 100:.2f}%")

# Optional: print full classification report
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating on custom_snowy_25.npy: 100%|██████████| 395/395 [00:19<00:00, 20.38it/s]



Evaluation Metrics for custom_snowy_25.npy
Accuracy : 81.86%
Precision: 91.78%
Recall   : 81.86%
F1 Score : 85.02%

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.82      0.88        60
           1       0.97      0.88      0.92       720
           2       0.91      0.86      0.89       750
           3       1.00      0.73      0.84       450
           4       0.99      0.77      0.86       660
           5       0.97      0.78      0.86       630
           6       0.95      0.70      0.80       150
           7       0.98      0.74      0.84       450
           8       0.99      0.74      0.84       450
           9       0.99      0.77      0.87       480
          10       0.98      0.75      0.85       660
          11       0.82      0.92      0.87       420
          12       1.00      0.87      0.93       690
          13       0.96      0.94      0.95       720
          14       0.67      0.90      0

In [11]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- CONFIG ---
# Base working directory
base_dir = "/content/drive/Shared drives/VisualCompGroupProject"

# Dataset path: just change this to test a different weather effect!
dataset_name = "custom_snowy_50.npy"  # Change to: custom_rainy.npy, custom_snowy.npy
dataset_path = os.path.join(base_dir, "SimpleMethodAugmentation", dataset_name)

# Path to model
model_path = os.path.join(base_dir, "resnet50_model.pth")

# Path to labels (must be aligned with test data)
y_test_path = os.path.join(base_dir, "SimpleMethodAugmentation", "labels_snowy_50.npy")  # Assumes labels are the same

# --- Dataset Wrapper ---
class NumpyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = int(self.labels[idx])
        return img, label

# --- Preprocessing ---
# These values are standard for ImageNet (which ResNet50 expects)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# --- Load Data ---
X_test = np.load(dataset_path)
y_test = np.load(y_test_path)

# Ensure dtype is correct
X_test = X_test.astype(np.uint8)

# Create DataLoader
test_dataset = NumpyDataset(X_test, y_test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 and match output classes
model = models.resnet50(pretrained=False)
num_classes = len(np.unique(y_test))
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# --- Evaluation ---
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc=f"Evaluating on {dataset_name}"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"\nEvaluation Metrics for {dataset_name}")
print(f"Accuracy : {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall   : {recall * 100:.2f}%")
print(f"F1 Score : {f1 * 100:.2f}%")

# Optional: print full classification report
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating on custom_snowy_50.npy: 100%|██████████| 395/395 [00:19<00:00, 20.26it/s]



Evaluation Metrics for custom_snowy_50.npy
Accuracy : 63.75%
Precision: 86.85%
Recall   : 63.75%
F1 Score : 70.25%

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.55      0.66        60
           1       0.94      0.71      0.81       720
           2       0.82      0.67      0.73       750
           3       1.00      0.49      0.66       450
           4       0.96      0.53      0.69       660
           5       0.93      0.53      0.68       630
           6       0.87      0.49      0.62       150
           7       0.95      0.51      0.66       450
           8       0.95      0.47      0.63       450
           9       0.97      0.60      0.74       480
          10       0.98      0.50      0.66       660
          11       0.66      0.80      0.72       420
          12       1.00      0.74      0.85       690
          13       0.93      0.86      0.89       720
          14       0.48      0.83      0

# Rain

In [12]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- CONFIG ---
# Base working directory
base_dir = "/content/drive/Shared drives/VisualCompGroupProject"

# Dataset path: just change this to test a different weather effect!
dataset_name = "custom_rainy_25.npy"  # Change to: custom_rainy.npy, custom_snowy.npy
dataset_path = os.path.join(base_dir, "SimpleMethodAugmentation", dataset_name)

# Path to model
model_path = os.path.join(base_dir, "resnet50_model.pth")

# Path to labels (must be aligned with test data)
y_test_path = os.path.join(base_dir, "SimpleMethodAugmentation", "labels_rainy_25.npy")  # Assumes labels are the same

# --- Dataset Wrapper ---
class NumpyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = int(self.labels[idx])
        return img, label

# --- Preprocessing ---
# These values are standard for ImageNet (which ResNet50 expects)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# --- Load Data ---
X_test = np.load(dataset_path)
y_test = np.load(y_test_path)

# Ensure dtype is correct
X_test = X_test.astype(np.uint8)

# Create DataLoader
test_dataset = NumpyDataset(X_test, y_test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 and match output classes
model = models.resnet50(pretrained=False)
num_classes = len(np.unique(y_test))
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# --- Evaluation ---
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc=f"Evaluating on {dataset_name}"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"\nEvaluation Metrics for {dataset_name}")
print(f"Accuracy : {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall   : {recall * 100:.2f}%")
print(f"F1 Score : {f1 * 100:.2f}%")

# Optional: print full classification report
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Evaluating on custom_rainy_25.npy: 100%|██████████| 395/395 [00:19<00:00, 20.42it/s]



Evaluation Metrics for custom_rainy_25.npy
Accuracy : 83.10%
Precision: 90.12%
Recall   : 83.10%
F1 Score : 85.06%

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.78      0.82        60
           1       0.90      0.90      0.90       720
           2       0.90      0.85      0.88       750
           3       1.00      0.70      0.83       450
           4       0.97      0.80      0.88       660
           5       0.97      0.79      0.87       630
           6       0.97      0.75      0.85       150
           7       0.97      0.75      0.84       450
           8       1.00      0.78      0.87       450
           9       0.99      0.82      0.89       480
          10       0.99      0.77      0.87       660
          11       0.78      0.92      0.84       420
          12       1.00      0.91      0.95       690
          13       0.73      0.96      0.83       720
          14       0.71      0.94      0

In [6]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# --- CONFIG ---
# Base working directory
base_dir = "/content/drive/Shared drives/VisualCompGroupProject"

# Dataset path: just change this to test a different weather effect!
dataset_name = "custom_rainy_50.npy"  # Change to: custom_rainy.npy, custom_snowy.npy
dataset_path = os.path.join(base_dir, "SimpleMethodAugmentation", dataset_name)

# Path to model
model_path = os.path.join(base_dir, "resnet50_model.pth")

# Path to labels (must be aligned with test data)
y_test_path = os.path.join(base_dir, "SimpleMethodAugmentation", "labels_rainy_50.npy")  # Assumes labels are the same

# --- Dataset Wrapper ---
class NumpyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = int(self.labels[idx])
        return img, label

# --- Preprocessing ---
# These values are standard for ImageNet (which ResNet50 expects)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# --- Load Data ---
X_test = np.load(dataset_path)
y_test = np.load(y_test_path)

# Ensure dtype is correct
X_test = X_test.astype(np.uint8)

# Create DataLoader
test_dataset = NumpyDataset(X_test, y_test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 and match output classes
model = models.resnet50(pretrained=False)
num_classes = len(np.unique(y_test))
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# --- Evaluation ---
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc=f"Evaluating on {dataset_name}"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"\nEvaluation Metrics for {dataset_name}")
print(f"Accuracy : {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall   : {recall * 100:.2f}%")
print(f"F1 Score : {f1 * 100:.2f}%")

# Optional: print full classification report
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Evaluating on custom_rainy_50.npy: 100%|██████████| 395/395 [00:19<00:00, 20.59it/s]

Accuracy on custom_rainy_50.npy: 65.95%
